### Урок 2. #Профилирование пользователей. Сегментация аудитории: unsupervised learning (clustering, LDA/ARTM), supervised (multi/binary classification)#

In [329]:
import pandas as pd

In [330]:
news = pd.read_csv("articles.csv")
print(news.shape)
news.head(3)

(27000, 2)


,doc_id,title
0,6,Заместитель председателяnправительства РФnСерг...
1,4896,Матч 1/16 финала Кубка России по футболу был п...
2,4897,Форвард «Авангарда» Томаш Заборский прокоммент...


In [331]:
users = pd.read_csv("users_articles.csv")
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


### 1. Получаем векторные представления новостей

In [332]:
#from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary

In [333]:
#предобработка текстов
import re
import numpy as np
from nltk.corpus import stopwords
#from nltk.tokenize import word_tokenize

from razdel import tokenize # https://github.com/natasha/razdel
#!pip install razdel

import pymorphy2  # pip install pymorphy2

In [334]:
# import nltk
# import ssl

# try:
#     _create_unverified_https_context = ssl._create_unverified_context
# except AttributeError:
#     pass
# else:
#     ssl._create_default_https_context = _create_unverified_https_context

# nltk.download()

In [335]:
stopword_ru = stopwords.words('russian')
len(stopword_ru)

morph = pymorphy2.MorphAnalyzer()

In [336]:
with open('stopwords.txt') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]
stopword_ru += additional_stopwords
len(stopword_ru)

776

In [337]:
def clean_text(text):
    '''
    очистка текста
    
    на выходе очищеный текст
    
    '''
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    
    #tokens = list(tokenize(text))
    #words = [_.text for _ in tokens]
    #words = [w for w in words if w not in stopword_ru]
    
    #return " ".join(words)
    return text

cache = {}

def lemmatization(text):
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист отлемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w)>1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords=[i for i in words_lem if not i in stopword_ru] # [6]
    
    return words_lem_without_stopwords

In [338]:
%%time
#Запускаем очистку текста. Будет долго...
news['title'] = news['title'].apply(lambda x: clean_text(x), 1)

CPU times: user 32.1 s, sys: 193 ms, total: 32.2 s
Wall time: 32.3 s


In [339]:
%%time
#Запускаем лемматизацию текста. Будет очень долго...
news['title'] = news['title'].apply(lambda x: lemmatization(x), 1)

CPU times: user 5min 1s, sys: 562 ms, total: 5min 2s
Wall time: 5min 2s


А теперь в 3 строчки обучим нашу модель

In [340]:
#сформируем список наших текстов, разбив еще и на пробелы
texts = [t for t in news['title'].values]

# Create a corpus from a list of texts
common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

Что такое common_dictionary и как он выглядит

In [341]:
common_dictionary[10]

'ватутин'

Все просто - это словарь наших слов

Запускаем обучение

In [342]:
%%time
from gensim.models import LdaModel
# Train the model on the corpus.
lda = LdaModel(common_corpus, num_topics=25, id2word=common_dictionary)#, passes=10)

CPU times: user 1min 5s, sys: 4.05 s, total: 1min 9s
Wall time: 45.6 s


In [343]:
from gensim.test.utils import datapath
# Save model to disk.
temp_file = datapath("model.lda")
lda.save(temp_file)

# Load a potentially pretrained model from disk.
lda = LdaModel.load(temp_file)

Обучили модель. Теперь 2 вопроса:

1. как выглядят наши темы
2. как получить для документа вектор значений (вероятности принадлежности каждой теме)

In [344]:
# Create a new corpus, made of previously unseen documents.
other_texts = [t for t in news['title'].iloc[:3]]
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]

unseen_doc = other_corpus[2]
print(other_texts[2])
lda[unseen_doc] 

['форвард', 'авангард', 'томаш', 'заборский', 'прокомментировать', 'игра', 'команда', 'матч', 'чемпионат', 'кхл', 'против', 'атланта', 'nnnn', 'плохой', 'матч', 'нижний', 'новгород', 'против', 'торпедо', 'настраиваться', 'первый', 'минута', 'включиться', 'заборский', 'получиться', 'забросить', 'быстрый', 'гол', 'задать', 'хороший', 'темп', 'поединок', 'играть', 'хороший', 'сторона', 'пять', 'очко', 'выезд', 'девять', 'хороший']


[(2, 0.16236429),
 (10, 0.032250214),
 (14, 0.07372059),
 (18, 0.54678),
 (19, 0.16358669)]

In [345]:
x=lda.show_topics(num_topics=25, num_words=7,formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

#Below Code Prints Only Words 
for topic,words in topics_words:
    print("topic_{}: ".format(topic)+" ".join(words))

topic_0: украина военный украинский nn восток гражданин киев
topic_1: налог мвд собственность отель наказание приговор перевод
topic_2: рынок рост млн банк статья решение новый
topic_3: продукция пограничный уверять приобретать открыться сопровождать шутка
topic_4: операция пациент пострадать произойти район заявлять лечение
topic_5: гражданин закон законопроект ракета парламент доклад запрет
topic_6: планета министерство космос расчёт китайский китай запустить
topic_7: греция майкл выручка челябинский греческий комиссар демократия
topic_8: товар оплата гражданство повышаться часовой дорого датский
topic_9: снижение обращение мужчина офицер железный таиланд терминал
topic_10: писать доллар рейс ступень аэропорт рейтинг виза
topic_11: путин владимир пенсия глава писать кремль пресссекретарить
topic_12: млрд турция учреждение объём энергия су развивающийся
topic_13: общество производить вино этаж департамент лауреат предупредить
topic_14: проект москва nn университет русский день участни

Очень неплохо - большинство тем вполне можно описать о чем они

Давайте напишем функцию, которая будет нам возвращать векторное представление новости

In [346]:
#text = news['title'].iloc[0]

def get_lda_vector(text):
    unseen_doc = common_dictionary.doc2bow(text)
    lda_tuple = lda[unseen_doc]
    not_null_topics = dict(zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))

    output_vector = []
    for i in range(25):
        if i not in not_null_topics:
            output_vector.append(0)
        else:
            output_vector.append(not_null_topics[i])
    return np.array(output_vector)

In [347]:
topic_matrix = pd.DataFrame([get_lda_vector(text) for text in news['title'].values])
topic_matrix.columns = ['topic_{}'.format(i) for i in range(25)]
topic_matrix['doc_id'] = news['doc_id'].values
topic_matrix = topic_matrix[['doc_id']+['topic_{}'.format(i) for i in range(25)]]
topic_matrix.head(5)

,doc_id,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,6,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.873195,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
1,4896,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.738570,0.0,0.000000,0.237211,0.000000,0.000000,0.000000,0.0,0.0
2,4897,0.0,0.000000,0.16365,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.545608,0.163641,0.000000,0.000000,0.000000,0.0,0.0
3,4898,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.589605,0.049050,0.225657,0.000000,0.125133,0.0,0.0
4,4899,0.0,0.395873,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.104702,0.0,0.000000,0.053572,0.000000,0.039401,0.000000,0.0,0.0


Прекрасно, мы получили вектора наших новостей! И даже умеем интерпретировать получившиеся темы.

Можно двигаться далее

### Следующий шаг - векторные представления пользователей

In [348]:
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


### Подготовим tfidf для статей пользователей

In [349]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [350]:
tfidf = TfidfVectorizer(max_features=15000)
tfidf.fit(users['articles'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=15000,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [351]:
idf = pd.DataFrame(tfidf.idf_, index=tfidf.get_feature_names(), columns=["idf"]) 
idf.sort_values(by=['idf'])

,idf
321929,7.096950
323233,7.096950
324181,7.096950
321850,7.096950
323631,7.154108
...,...
513087,9.294175
513085,9.294175
513083,9.294175
513094,9.294175


In [352]:
doc_dict = dict(zip(topic_matrix['doc_id'].values, topic_matrix[['topic_{}'.format(i) for i in range(25)]].values))

In [353]:
doc_dict[293622]

array([0.        , 0.        , 0.06896462, 0.        , 0.03421551,
       0.04968504, 0.10160048, 0.03472021, 0.        , 0.        ,
       0.        , 0.11997338, 0.        , 0.        , 0.17824621,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.17258951, 0.20951281, 0.02029782])

In [354]:
user_articles_list = users['articles'].iloc[33]



**У меня возникла странная проблема - в tfidf не учлись статьи с id < 10, хотя они встречались в датафрейме, на котором вычислялись в tfidf. Я некоторое время птылася разобраться с проблемой, но потом просто поставил заглушку чтобы пропускать статьи с индексом менее 10, во всей выборке их встретилось всего несколько штук. Возможно вы подскажете в чем может быть проблема?**

In [355]:
target = pd.read_csv("users_churn.csv")
target.head(3)

,uid,churn
0,u107120,0
1,u102277,0
2,u102444,0


In [368]:
result = pd.DataFrame(columns=['type', 'roc_auc', 'precision', 'recall', 'f-score'])

In [372]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

mode = [['no_idf', 'mean'],
        ['no_idf', 'median'],
        ['no_idf', 'max'],
        ['idf', 'mean'],
        ['idf', 'median'],
        ['idf', 'max']]



for m in mode:
    
    def get_user_embedding(user_articles_list):
        user_articles_list = eval(user_articles_list)
        
        if m[0] == 'idf':
            user_vector = np.array([(doc_dict[doc_id]*idf.loc[idf.index==str(doc_id), 'idf'][0]) for doc_id in user_articles_list if doc_id >= 10])
        else:
            user_vector = np.array([(doc_dict[doc_id]) for doc_id in user_articles_list])
            
        if m[1] == 'mean':
            user_vector = np.mean(user_vector, 0)
        elif m[1] == 'median':
            user_vector = np.median(user_vector, 0)
        else:
            user_vector = np.max(user_vector, 0)
            
        
        return user_vector

    user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x), 1)])
    user_embeddings.columns = ['topic_{}'.format(i) for i in range(25)]
    user_embeddings['uid'] = users['uid'].values
    user_embeddings = user_embeddings[['uid']+['topic_{}'.format(i) for i in range(25)]]

    X = pd.merge(user_embeddings, target, 'left')

    X_train, X_test, y_train, y_test = train_test_split(X[['topic_{}'.format(i) for i in range(25)]], 
                                                        X['churn'], random_state=0)

    logreg = LogisticRegression()
    logreg.fit(X_train, y_train)

    preds = logreg.predict_proba(X_test)[:, 1]

    precision, recall, thresholds = precision_recall_curve(y_test, preds)
    fscore = (2 * precision * recall) / (precision + recall)
    # locate the index of the largest f score
    fscore = fscore[~np.isnan(fscore)]
    ix = np.argmax(fscore)
    
    result = result.append({'type':str(m[0] + ' ' + m[1]),
                   'roc_auc':roc_auc_score(y_test, preds),
                   'precision':precision[ix],
                   'recall':recall[ix],
                   'f-score':fscore[ix]}, ignore_index=True)
    


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    

In [373]:
result

,type,roc_auc,precision,recall,f-score
0,no_idf mean,0.919714,0.599222,0.628571,0.613546
1,no_idf median,0.956897,0.752294,0.669388,0.708423
2,no_idf max,0.963296,0.702602,0.771429,0.735409
3,idf mean,0.980738,0.817427,0.804082,0.810700
4,idf median,0.979810,0.846491,0.787755,0.816068
5,idf max,0.957298,0.785388,0.702041,0.741379


### Домашнее задание

1. Самостоятельно разобраться с тем, что такое tfidf (документация https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html и еще - https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction)
2. Модифицировать код функции get_user_embedding таким образом, чтобы считалось не среднее (как в примере np.mean), а медиана. Применить такое преобразование к данным, обучить модель прогнозирования оттока и посчитать метрики качества и сохранить их: roc auc, precision/recall/f_score (для 3 последних - подобрать оптимальный порог с помощью precision_recall_curve, как это делалось на уроке)
3. Повторить п.2, но используя уже не медиану, а max
4. (опциональное, если очень хочется) Воспользовавшись полученными знаниями из п.1, повторить пункт 2, но уже взвешивая новости по tfidf (подсказка: нужно получить веса-коэффициенты для каждого документа. Не все документы одинаково информативны и несут какой-то положительный сигнал). Подсказка 2 - нужен именно idf, как вес.
5. Сформировать на выходе единую таблицу, сравнивающую качество 3 разных метода получения эмбедингов пользователей: mean, median, max, idf_mean по метрикам roc_auc, precision, recall, f_score
6. Сделать самостоятельные выводы и предположения о том, почему тот или ной способ оказался эффективнее остальных

### Результаты:
    - применил idf в качестве весов для статей, умножая вектор тем статьи на idf. Таким образом статьи, которые читались чаще будут иметь значения вектора меньше, чем те статьи, которые читались реже. 
    - без применения весов idf лучший результат дало применение метода max. Это логично - ведь если возникнет ситуация когда пользователь прочитал две статьи, одна имеет по теме1 большой вес, другая по теме2, то использование усредняющих показателей в итоге усреднит значения тем для статей и по сути уберет полезную информацию о пользователе, тогда как параметр max создаст наиболее характерный вектор.
    - при применении весов idf результаты работы модели ожидаемо возросли, при этом отметим, что лучший реультат досигнут при использовании усредняющих показателей - median или mean, смотря по какому параметру оценивать. Такой результат наблюдается в связи с тем, что теперь вектора статей имеют веса, и больше нельзя в столбце просто выбирать максимальный. 